In [ ]:
#Luca: We must tile all the WSI before proceeding with either segmentation or WSI-level. This notebook does the preprocessing 


#Segmentation Pipeline
    # Tile
    # Give tile name based on coordinate
    # Extract polygon annotation
    # Generate Masks based on Polygon Annotation
    # Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
    # Separate tiles into BG, Inf and Healthy folders
#WSI-level Pipeline
    # Tile
    # Remove BG tiles using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
        # and name them sequentially

In [1]:
import os
#import os.path
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import pyvips as Vips
from tqdm import tqdm
from utils import vips_utils, normalize
from torchvision import transforms, utils
import time
import torchvision.models as models
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from PIL import Image, ImageFile
import statistics
from typing import Optional, Tuple
import pylibczi
from pylibczi import CziScene
import czifile
from czifile import CziFile 
import xml.etree.ElementTree as ET
import argparse
import gc 
import psutil
import resource
import platform
import pickle
import xmltodict
import time
import matplotlib.path as mpath
from skimage.draw import polygon
import cv2
import copy
import shutil

import torch
import random
import torchvision
import torch.nn as nn
from skimage import io, transform
### PARAMETERS

In [2]:
TILE_SIZE = 512

WSI_DIR = '/cache/Inf_May23_dataset/'
SAVE_DIR = '/cache/S23_Infarct/patched_'+str(TILE_SIZE)+'/'
CZ_DIR = '/cache/S23_Infarct/annotation/'
MASK_DIR = '/cache/S23_Infarct/masks/'

SEGMENTATION_TILE_DIR = '/cache/S23_Infarct/seg_data_'+str(TILE_SIZE)+'/'
WSI_TILE_DIR = '/cache/S23_Infarct/wsi_level_data_'+str(TILE_SIZE)+'/'

tile_czi = True
tile_svs = True

In [3]:
if not os.path.exists(WSI_DIR):
    print("WSI folder does not exist, script should stop now")
else:
    if not os.path.exists(SEGMENTATION_TILE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SEGMENTATION_TILE_DIR)
        
    if not os.path.exists(WSI_TILE_DIR):
        print("Tile folder for WSI-level you provided us does not exist, being created now...")
        os.makedirs(WSI_TILE_DIR)

    if not os.path.exists(SAVE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SAVE_DIR)

    if not os.path.exists(CZ_DIR):
        print("Annotation folder you provided us does not exist, being created now...")
        os.makedirs(CZ_DIR)

    if not os.path.exists(MASK_DIR):
        print("Mask folder you provided us does not exist, being created now...")
        os.makedirs(MASK_DIR)

    print("Found WSI folder... ")
    wsi_slides = os.listdir(WSI_DIR)
    imagenames = sorted(wsi_slides)
    print("All WSIs in wsi_dir: ")
    print(imagenames)

Found WSI folder... 
All WSIs in wsi_dir: 
['NA-5029-16_HE.svs', 'NA-5029-17_HE.svs', 'NA-5029-18_HE.svs', 'NA5004-16_HE.svs', 'NA5004-17_HE.svs', 'NA5004-18_HE.svs', 'NA5007-16_HE.svs', 'NA5007-17_HE.svs', 'NA5007-18_HE.svs', 'NA5009-16_HE.svs', 'NA5009-17_HE.svs', 'NA5009-18_HE.svs', 'NA5031-16_HE.svs', 'NA5031-17_HE.svs', 'NA5031-18_HE.svs', 'NA5041-16_HE.svs', 'NA5041-17_HE.svs', 'NA5041-18_HE.svs', 'NA5045-16_HE.svs', 'NA5045-17_HE.svs', 'NA5045-18_HE.svs', 'NA5051-16_HE.czi', 'NA5051-17_HE.czi', 'NA5051-18_HE.czi', 'NA5057-16_HE.svs', 'NA5057-17_HE.svs', 'NA5057-18_HE.svs', 'NA5063-16_HE.czi', 'NA5063-17_HE.czi', 'NA5063-18_HE.czi', 'NA5077-16_HE.czi', 'NA5077-17_HE.czi', 'NA5077-18_HE.czi', 'NA5085-16_HE.czi', 'NA5085-17_HE.czi', 'NA5085-18_HE.czi', 'NA5089-16_HE.czi', 'NA5089-17_HE.czi', 'NA5089-18_HE.czi', 'NA5090-16_HE.czi', 'NA5090-17_HE.czi', 'NA5090-18_HE.czi', 'NA5091-16_HE.czi', 'NA5091-17_HE.czi', 'NA5091-18_HE.czi', 'NA5093-16_HE.czi', 'NA5093-17_HE.czi', 'NA5093-18_HE

In [4]:
def grabCZI(path, verbose = False):
    img = czifile.imread(path)
    if verbose:
        print(img.shape)
        print(img)
    
    img = np.array(img, dtype = np.uint8)
    
    scenes = img.shape[0]
    time = img.shape[1]
    height = img.shape[2]
    width = img.shape[3]
    channels = img.shape[4]
    
    
    img = img.reshape((height, width, channels))
    if verbose:
        print(img)
        print(img.shape) 
        
    dtype_to_format = {
        'uint8': 'uchar',
        'int8': 'char',
        'uint16': 'ushort',
        'int16': 'short',
        'uint32': 'uint',
        'int32': 'int',
        'float32': 'float',
        'float64': 'double',
        'complex64': 'complex',
        'complex128': 'dpcomplex',
    }
    
    ###codes from numpy2vips
    height, width, bands = img.shape
    img = img.reshape(width * height * bands)
    vips = Vips.Image.new_from_memory(img.data, width, height, bands,
                                      dtype_to_format['uint8'])
    try: 
        del img, height, width, bands
        gc.collect()
    except: 
        pass
    
    return vips

In [ ]:
###### SEGMENTATION / WSI-level Pipeline - Tile

print("Starting tiling....")
for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
        
        print("Pre resize: ", vips_img.height, "x", vips_img.width)
        
        if NAID in ['NA5009-16_HE', 'NA5009-17_HE', 'NA5009-18_HE']:
            pass
            #vips_img = vips_img.resize(0.25)
        else:
            print("Resizing small case")
            vips_img = vips_img.resize(2)
            
        print("Post resize: ", vips_img.height, "x", vips_img.width)
            
        print("Loaded Image: " + WSI_DIR + imagename)

        if tile_svs:
        
            vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                     , SAVE_DIR, tile_size = TILE_SIZE)
            print("Done Tiling: ", WSI_DIR + imagename)
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            vips_img = grabCZI(WSI_DIR + imagename)
            print("Loaded Image: " + WSI_DIR + imagename)
            
            print("Pre resize: ", vips_img.height, "x", vips_img.width)
            
            if tile_czi:
        
                vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                         , SAVE_DIR, tile_size = TILE_SIZE)

                print("Done Tiling: ", WSI_DIR + imagename)
                del vips_img
                gc.collect()
                print("Finish Delete", WSI_DIR + imagename)
        except:
            print("Error in tiling")
        
        

    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
    
    try: 
        del vips_img 
        gc.collect()
    except:
        pass
    
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

Starting tiling....


  0%|          | 0/69 [00:00<?, ?it/s]

Loading NA-5029-16_HE.svs  ......
Pre resize:  34033 x 45816
Resizing small case
Post resize:  68066 x 91632
Loaded Image: /cache/Inf_May23_dataset/NA-5029-16_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-16_HE


In [5]:
###### SEGMENTATION - Give tile name based on coordinate

print("About to change names to add coordinates")
for case_folder in sorted(os.listdir(SAVE_DIR)):
    NAID = case_folder
    print('Processing NAID: ', NAID)
    for tile_folder in sorted(os.listdir(SAVE_DIR+NAID+'/0/')):
        # folder_level == y axis distance determinant
        # file_level == x axis distance determinant
        y0 = int(tile_folder)*TILE_SIZE
        y1 = (int(tile_folder)+1)*TILE_SIZE
        #print("Y axis = (", y0,',',y1,')')
        for tile_file in sorted(os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder+'/')):
            x0 = int(tile_file.split('.')[0])*TILE_SIZE
            x1 = (int(tile_file.split('.')[0])+1)*TILE_SIZE
            #print("X axis = (", x0,',',x1,')')
            #print("Renaming ", SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file)
            os.rename(SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file, 
                      SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+tile_file)
            #print("Renamed into ", SAVE_DIR+NAID+'/0/'+tile_folder+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+t

About to change names to add coordinates
Processing NAID:  NA-5029-16_HE
Processing NAID:  NA-5029-17_HE
Processing NAID:  NA-5029-18_HE
Processing NAID:  NA5004-16_HE
Processing NAID:  NA5004-17_HE
Processing NAID:  NA5004-18_HE
Processing NAID:  NA5007-16_HE
Processing NAID:  NA5007-17_HE
Processing NAID:  NA5007-18_HE
Processing NAID:  NA5009-16_HE
Processing NAID:  NA5009-17_HE
Processing NAID:  NA5009-18_HE
Processing NAID:  NA5031-16_HE
Processing NAID:  NA5031-17_HE
Processing NAID:  NA5031-18_HE
Processing NAID:  NA5041-16_HE
Processing NAID:  NA5041-17_HE
Processing NAID:  NA5041-18_HE
Processing NAID:  NA5045-16_HE
Processing NAID:  NA5045-17_HE
Processing NAID:  NA5045-18_HE
Processing NAID:  NA5051-16_HE
Processing NAID:  NA5051-17_HE
Processing NAID:  NA5051-18_HE
Processing NAID:  NA5057-16_HE
Processing NAID:  NA5057-17_HE
Processing NAID:  NA5057-18_HE
Processing NAID:  NA5063-16_HE
Processing NAID:  NA5063-17_HE
Processing NAID:  NA5063-18_HE
Processing NAID:  NA5077-1

In [6]:
###### SEGMENTATION - Extract polygon annotation

for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        pass
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            czifile = pylibczi.CziFile(WSI_DIR + imagename, metafile_out = CZ_DIR + NAID + '.cz',use_pylibczi=True, verbose=True)
            czifile.read_meta()
            
            tree = ET.parse(CZ_DIR + NAID + '.cz') 
            root = tree.getroot() 
            
            tree.write(CZ_DIR + NAID + '.xml')
        except:
            print("Error in extracting annotation")
            
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

 35%|███▍      | 24/69 [00:00<00:00, 179.24it/s]

processed in  3.337860107421875e-06  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  1.430511474609375e-06  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed i

 61%|██████    | 42/69 [00:00<00:00, 40.97it/s] 

processed in  0.037004709243774414  seconds
____________________________________________
Loading NA5091-16_HE.czi  ......
processed in  0.040390729904174805  seconds
____________________________________________
Loading NA5091-17_HE.czi  ......
processed in  0.04259204864501953  seconds
____________________________________________
Loading NA5091-18_HE.czi  ......
processed in  0.04012298583984375  seconds
____________________________________________
Loading NA5093-16_HE.czi  ......
processed in  0.0407557487487793  seconds
____________________________________________
Loading NA5093-17_HE.czi  ......
processed in  0.043538808822631836  seconds
____________________________________________
Loading NA5093-18_HE.czi  ......
processed in  0.11716198921203613  seconds
____________________________________________
Loading NA5095-16_HE.czi  ......
processed in  0.04244589805603027  seconds
____________________________________________
Loading NA5095-17_HE.czi  ......


 74%|███████▍  | 51/69 [00:01<00:00, 31.73it/s]

...

processed in  0.04429054260253906  seconds
____________________________________________
Loading NA5114-17_HE.czi  ......
processed in  0.1206672191619873  seconds
____________________________________________
Loading NA5114-18_HE.czi  ......
processed in  0.04523038864135742  seconds
____________________________________________
Loading NA5116-16_HE.czi  ......


 90%|████████▉ | 62/69 [00:01<00:00, 26.99it/s]

processed in  0.03956413269042969  seconds
____________________________________________
Loading NA5116-17_HE.czi  ......
processed in  0.03751325607299805  seconds
____________________________________________
Loading NA5116-18_HE.czi  ......
processed in  0.038360595703125  seconds
____________________________________________
Loading NA5137-16_HE.czi  ......
processed in  0.038420915603637695  seconds
____________________________________________
Loading NA5137-17_HE.czi  ......
processed in  0.039307355880737305  seconds
____________________________________________
Loading NA5137-18_HE.czi  ......
processed in  0.03811335563659668  seconds
____________________________________________
processed in  3.5762786865234375e-06  seconds
____________________________________________
processed in  9.5367431640625e-07  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
Loading NA5161-16_HE.czi  ......


100%|██████████| 69/69 [00:02<00:00, 32.63it/s]

processed in  0.03740048408508301  seconds
____________________________________________
Loading NA5161-17_HE.czi  ......
processed in  0.11650562286376953  seconds
____________________________________________
Loading NA5161-18_HE.czi  ......
processed in  0.03840994834899902  seconds
____________________________________________


In [11]:
###### SEGMENTATION - Generate Masks based on Polygon Annotation
def grab_svs_annot(path):
    doc = xmltodict.parse(open(path, 'r', encoding='utf-8').read())
    
    all_annot = []
    if type(doc['Annotations']['Annotation']['Regions']['Region']) == list:
        for region_idx in range(len(doc['Annotations']['Annotation']['Regions']['Region'])):
            coords = []
            for annot in doc['Annotations']['Annotation']['Regions']['Region'][region_idx]['Vertices']['Vertex']:
                coord = [int(annot['@Y']),int(annot['@X'])]

                coords.append(coord)
            all_annot.append(coords)
    else:
        coords = []
        for annot in doc['Annotations']['Annotation']['Regions']['Region']['Vertices']['Vertex']:
            coord = [int(annot['@Y']),int(annot['@X'])]

            coords.append(coord)
        all_annot.append(coords)
    
    return all_annot

#from: https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
def findkeys(node, kv):
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
               yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

def grab_czi_annot(path):
    doc = xmltodict.parse(open(path, 'r', encoding='utf-8').read())
    
    amt_annot = len(list(findkeys(doc,'Points')))
    
    all_annot = []
    coords = []
    
    #print("AMOUNT ANNOT = ", amt_annot)
    
    for annot_idx in range(amt_annot):
        coords = []
        all_cord = list(findkeys(doc,'Points'))[annot_idx].split(' ')
        isGood = False
        for xy in all_cord:
            print(xy)
            if int(float(xy.split(',')[1])) < 0 or int(float(xy.split(',')[0])) < 0:
                pass
                #print("Passed on negative coordinates")
            else:
                coord = [int(float(xy.split(',')[1])),int(float(xy.split(',')[0]))]
                isGood = True
                
                coords.append(coord)
        if isGood:
            all_annot.append(coords)
    
    return all_annot



def create_polygon_mask(vertex_coordinates, mask):
    
    rr, cc = polygon(vertex_coordinates[:,0],vertex_coordinates[:,1],mask.shape)
    mask[rr,cc] = 1
    
    return mask

gt_df = pd.read_csv('/cache/Infarct/GNN_Hybrid/gt.csv')

for imagename in tqdm(imagenames[:]):
    start = time.time()

    wsi_gt = gt_df[gt_df.cases == imagename].iat[0,2]
    
    if wsi_gt == 1:
        if imagename.split('.')[-1] == 'svs':
            NAID = imagename.split('.')[0]
            print("Loading", imagename, " ......")
            if True: 
                vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
                print("Loaded Image: " + WSI_DIR + imagename)

                dimension = [vips_img.height, vips_img.width]

                all_annot = grab_svs_annot(CZ_DIR + NAID + '.xml')

                mask = np.zeros(dimension,'uint8')
                for coords in all_annot:
                    coords = np.asarray(coords,dtype=np.uint8)

                    mask = create_polygon_mask(coords, mask)

                #mask = np.packbits(mask,axis=None)
                print("Binarized mask")

                np.save(MASK_DIR+NAID+'.npy',mask) 

                print("processed in ", time.time()-start," seconds")
                print("____________________________________________")

            else:
                print("Error in generating masks from polygon for ", NAID)


        elif imagename.split('.')[-1] == 'czi':
            ### TO DO: CHECK IF GENERALIZABLE TO ALL CASES, INCLUDING WITH SINGLE OR MULTIPLE TRACES

            NAID = imagename.split('.')[0]
            print("Loading", imagename, " ......")
            if True: 
                vips_img = grabCZI(WSI_DIR + imagename)
                print("Loaded Image: " + WSI_DIR + imagename)

                dimension = [vips_img.height, vips_img.width]
                all_annot = grab_czi_annot(CZ_DIR + NAID + '.xml')

                mask = np.zeros(dimension,'uint8')
                for coords in all_annot:
                    coords = np.asarray(coords,dtype=np.uint8)

                    mask = create_polygon_mask(coords, mask)

                #mask = np.packbits(mask,axis=None)
                print("Binarized mask")

                np.save(MASK_DIR+NAID+'.npy',mask) 

                print("processed in ", time.time()-start," seconds")
                print("____________________________________________")
            else:
                print("Error in generating masks from polygon for ", NAID)
        
        
    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
    
    if wsi_gt == 1:
        del vips_img,coords,mask
        gc.collect()

  0%|          | 0/69 [00:00<?, ?it/s]

Skipped, NA-5029-16_HE.svs . This file is either not .czi or .svs, or not the file assigned
Skipped, NA-5029-17_HE.svs . This file is either not .czi or .svs, or not the file assigned
Loading NA-5029-18_HE.svs  ......
Loaded Image: /cache/Inf_May23_dataset/NA-5029-18_HE.svs
[[253 152]
 [  2 152]
 [ 13 157]
 ...
 [243 163]
 [248 163]
 [253 152]]
941
[[36 82]
 [36 71]
 [36 67]
 ...
 [46 82]
 [43 82]
 [36 82]]
596
[[ 58 126]
 [ 27  95]
 [  6  84]
 ...
 [ 48 158]
 [ 48 147]
 [ 58 126]]
848
[[200 214]
 [200 183]
 [200 151]
 [200 130]
 [200 109]
 [200  88]
 [200  67]
 [200  46]
 [200  25]
 [200  15]
 [200   5]
 [200 240]
 [200 229]
 [200 219]
 [200 208]
 [200 187]
 [189 166]
 [179 145]
 [179 135]
 [179 124]
 [168 103]
 [168  93]
 [168  72]
 [147  30]
 [147  19]
 [147 254]
 [147 244]
 [147 233]
 [147 212]
 [147 202]
 [137 191]
 [137 181]
 [137 170]
 [137 160]
 [137 149]
 [137 139]
 [137 129]
 [137 118]
 [137 108]
 [137  87]
 [147  55]
 [158  34]
 [179  13]
 [189   3]
 [200 238]
 [200 227]
 [2

  4%|▍         | 3/69 [00:02<01:00,  1.10it/s]

processed in  2.6592185497283936  seconds
____________________________________________
Skipped, NA5004-16_HE.svs . This file is either not .czi or .svs, or not the file assigned
Skipped, NA5004-17_HE.svs . This file is either not .czi or .svs, or not the file assigned
Loading NA5004-18_HE.svs  ......
Loaded Image: /cache/Inf_May23_dataset/NA5004-18_HE.svs
[[230  79]
 [229  79]
 [229  78]
 ...
 [234  76]
 [234  75]
 [230  79]]
552
Binarized mask


  9%|▊         | 6/69 [00:03<00:32,  1.92it/s]

...

 30%|███       | 21/69 [00:10<00:20,  2.32it/s]

processed in  0.8666322231292725  seconds
____________________________________________
Skipped, NA5051-16_HE.czi . This file is either not .czi or .svs, or not the file assigned
Skipped, NA5051-17_HE.czi . This file is either not .czi or .svs, or not the file assigned
Loading NA5051-18_HE.czi  ......


 33%|███▎      | 23/69 [00:33<01:06,  1.44s/it]

Loaded Image: /cache/Inf_May23_dataset/NA5051-18_HE.czi
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
0


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
###### SEGMENTATION - Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)

MODEL_SEG_DIR = './patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt'

seg_model = torchvision.models.resnet18()
seg_model.fc = nn.Linear(512, 3)

checkpoint = torch.load(MODEL_SEG_DIR)
seg_model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seg_model.to(device)


for NAID in os.listdir(SAVE_DIR):
    os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/BG/')
    for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
        for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):
            seg_model.train(False)
            
            img = Image.open(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile)
            
            img_tensor = transforms.ToTensor()(img)
            img_tensor = transforms.Normalize(mean=[0.4409763317567454, 0.4016568471536302, 0.4988298669112181],
                             std=[0.31297803931100737, 0.2990562933047881, 0.33747493782548915])(img_tensor)
            img_tensor = torch.reshape(img_tensor,(1,3,512,512))
            img_tensor = img_tensor.cuda()
            
            predict = seg_model(img_tensor)
            preds = F.sigmoid(predict)
            _, indices = torch.max(predict.data, 1) # indices = 0:Background, 1:WM, 2:GM
            indices = indices.type(torch.uint8)
            running_seg =  indices.data.cpu()
            
            
            if running_seg == 0:
                shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/BG/'+tile)
            else:
                pass
                #print("bg removed")

In [ ]:
###### SEGMENTATION - Separate tiles into BG, Inf and Healthy folders

for NAID in os.listdir(SAVE_DIR):
    proceed = True
    
    if os.path.isfile(WSI_DIR+NAID+'.svs'):
        vips_img = Vips.Image.new_from_file(WSI_DIR+NAID+'.svs', level=0)
    elif os.path.isfile(WSI_DIR+NAID+'.czi'):
        vips_img = grabCZI(WSI_DIR+NAID+'.czi')
    else:
        print("Could not find WSI - it is not czi nor svs")
        proceed = False
    
    if proceed:
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Heal/')
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Inf/')
        
        img_dim = [vips_img.height,vips_img.width]
        mask = np.load(MASK_DIR+NAID+'.npy')
        print("Loaded mask at ", MASK_DIR+NAID+'.npy' ,", about to process...")

        im_size = img_dim[0]*img_dim[1]

        mask = np.unpackbits(mask,count=im_size).reshape(img_dim).view(bool)
        mask = mask.view(np.uint8)
        print("Unbinarized mask")
        
        
        for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
            for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):

                #file naming convention --> y0 x0 y1 x1
                y0 = tile.split('_')[0]
                x0 = tile.split('_')[1]
                y1 = tile.split('_')[2]
                x1 = tile.split('_')[3]
                
                if sum(sum(masky[y0:y1,x0:x1])) >= (512/3):
                    shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Inf/'+tile)
                else:
                    shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Heal/'+tile)
            
            
    del vips_img, mask
    gc.collect()